# 本章节均为Fluent Python中的重点内容

# 类型
## 内置类型
* 按照存储的数据类型：
    * 容器序列（存放不同类型的数据）：list、tuple、collections.deque
    * 扁平序列（只能存放一种类型的数据）：str、bytes、bytearray、memoryview和array.array。
* 按照能否被修改
    * 可变序列：list、bytearray、array.array、collections.deque和memoryview
    * 不可变序列：tuple、str、bytes

### 列表list

**列表推导**

In [3]:
symbols = "!@#$%^&*()"
code = []
for i in symbols:
    code.append(ord(i))
code

[33, 64, 35, 36, 37, 94, 38, 42, 40, 41]

In [7]:
codes = [ord(x) for x in symbols]
codes

[33, 64, 35, 36, 37, 94, 38, 42, 40, 41]

1. 简洁易读
2. for循环存在变量泄露的问题，如

In [14]:
i

')'

In [15]:
x

NameError: name 'x' is not defined

更多形式：


1. 支持多维和判断


In [46]:
x = ['♥️', '♠️', '♦️', '♣️']
y = [str(x) for x in range(2, 11)] + list("JQKA")
cards = [i+j for i in x for j in y]
black_cards = [i + j for i in x if i != "♠️" and i != "♣️" for j in y]
print(len(cards))
print(len(black_cards))

52
26


2. 生成器版：节省内存和性能，不利于重复使用

In [48]:
cards_generator = (i+j for i in x for j in y)
cards_generator

<generator object <genexpr> at 0x10c7597e0>

### 元组

元组拆包

In [63]:
# cell info
cellinfo = ("TDD", "30kHz", "100M", "4T4R")
rat, scs, bw, trx = cellinfo
print(trx)
*_, trx = cellinfo 
print(trx)

4T4R
4T4R


具名元组namedtuple

In [1]:
from collections import namedtuple
# 类名, 参数名列表
Cell = namedtuple('CellInfo', ['rat','scs', 'bw', 'trx'])
cellinfo = Cell("TDD", "30kHz", "100M", "4T4R")
print(cellinfo.trx)
print(cellinfo[3])

4T4R
4T4R


# dict & set

## 可散列的(hashable)
1. 实现`__hash__()`，要求同一个实例结果不变，`hash(a)==hash(a)`
2. 实现`__eq__()`，能够检验相等
3. 若`a==b`，则`hash(a)==hash(b)`

不可变的实例是可散列的，自定义对象默认可散列，由`id(instance)`获取。不建议自定义，特别是`__eq__`依赖可变的状态。

## dict

In [104]:
# 创建方式
suits = ["♠️", "♥️", "♣️", "♦️"]
shape_name = dict(spade="♠️", heart="♥️", club="♣️", diamond="♦️")
shape_name = dict(zip(["spade", "heart", "diamond", "club"], suits))

value_map = {"A":1, "2": 2, "3": 3, "4": 4, "5": 5,"6": 6,"7": 7,"8": 8,"9": 9,"10": 10, "J": 0.5, "Q": 0.5, "K":0.5}
value_map = {str(x): x for x in range(2, 11)}
value_map.update({"A": 1})
value_map.update(dict(zip(list("JQK"), [0.5] * 3)))
value_map = dict({str(x): x for x in range(2, 11)}, A=1, J=0.5, Q=0.5, K=0.5)
value_map = dict({str(x): x for x in range(2, 11)}, **{"A":1, "J":0.5, "Q":0.5, "K":0.5})
# dict(name=name, description=desc % count, points=points, parent_award=parent, **award_dict)

### 默认值&类型的dict.get(key, defaultvalue)
除了"JQK"还有王"joker"还有空白牌、商标牌、广告牌、规则牌，想要全都定义是很困难的，但规则是简单的：
凡是不在已有规则里的都是0.5点

In [107]:
comm_value_map = dict({str(x): x for x in range(2, 11)}, A=1)
print(comm_value_map.get("A", 0.5))
blank_card = ""
print(comm_value_map.get(blank_card, 0.5))

1
0.5


#### collections.defaultdict(default_factory)

In [113]:
from collections import defaultdict
comm_value_map = defaultdict(lambda: 0.5)
from functools import partial
comm_value_map = defaultdict(partial(float, 0.5))
# 伪代码，定义__missing__或default_factory对contain没有影响
"""
def __getitem__(self, item):
    if self.__contain__(item):
        return self[item]
    return self.__missing__(item)

def __missing__(self, item):
    if self.default_factory:
        return self.default_factory()
    raise KeyError(item)

def __contain__(self, item):
    return item in self.keys()
"""
comm_value_map[blank_card]
# 会使getdefault失效
print(comm_value_map.get(blank_card, 2))

0.5


### 只读dict: MappingProxyType

In [114]:
from types import MappingProxyType
value_map_readonly = MappingProxyType(comm_value_map)
print(value_map_readonly)
value_map_readonly["A"] = 0.5

defaultdict(..., {'': 0.5})


TypeError: 'mappingproxy' object does not support item assignment

In [115]:
# 改变原始值
comm_value_map["A"] = 0.5
print(value_map_readonly["A"])
comm_value_map["A"] = 1

0.5


### 其他变种
* OrderedDict: 按照添加顺序排序，但只用于遍历处理，key是以链的形式存储的，获取key位置的时间O(n)
* UserDict：python版的dict

In [121]:
from collections import OrderedDict
# 黑桃、红桃、梅花、方片
shape_name = OrderedDict(zip(["spade", "heart", "diamond", "club"], suits))
print(shape_name.keys())
print(shape_name)

odict_keys(['spade', 'heart', 'diamond', 'club'])
OrderedDict([('spade', '♠️'), ('heart', '♠️'), ('diamond', '♦️'), ('club', '♣️')])


### 散列表的内存占用巨大
* dict vs tuple vs namedtuple vs dataclass

In [168]:
from sys import getsizeof
# dict
spade_a_dict = {"suit": "♠️", "number": "A"}
spade_a_dict2 = {"suit": "♠️", "number": "A"}

# plain tuple
spade_a_tuple = ("♠️", "A")
spade_a_tuple2 = ("♠️", "A")

# namedtuple
from collections import namedtuple
CardNt = namedtuple("Card", ["suit", "number"])
spade_a_nt = CardNt("♠️", "A")
spade_a_nt2 = CardNt("♠️", "A")
# dataclass !不要新定义函数
from dataclasses import dataclass
@dataclass
class CardDc:
    suit: str
    number: str
    def __init__(self, suit, number, add="2") -> None:
        self.suit = suit
        self.number = number
        self.add = add
    def rankscore(self):
        return self.number
spade_a_dc = CardDc("♠️", "A", 3)
spade_a_dc2 = CardDc("♠️", "A")

print(spade_a_dc.add, spade_a_dc2.add)
# class
class CardPc:
    def __init__(self, suit, number) -> None:
        self.suit = suit
        self.number = number
    def rankscore(self):
        return self.number
spade_a_pc = CardPc("♠️", "A")
spade_a_pc2 = CardPc("♠️", "A")
print(f"dict size {getsizeof(spade_a_dict)}, hashable {hasattr(spade_a_dict, '__hash__')}, instance eq {spade_a_dict == spade_a_dict2}")
print(f"tuple size {getsizeof(spade_a_tuple)}, hashable {hasattr(spade_a_tuple, '__hash__')}, instance eq {spade_a_tuple == spade_a_tuple2}")
print(f"namedtuple size {getsizeof(spade_a_nt)}, hashable {hasattr(spade_a_nt, '__hash__')}, instance eq {spade_a_nt == spade_a_nt2}")
print(f"dataclass size {getsizeof(spade_a_dc)}, hashable {hasattr(spade_a_dc, '__hash__')}, instance eq {spade_a_dc == spade_a_dc2}")
print(f"class size {getsizeof(spade_a_pc)}, hashable {hasattr(spade_a_pc, '__hash__')}, instance eq {spade_a_pc == spade_a_pc2}")


3 2
dict size 184, hashable True, instance eq True
tuple size 56, hashable True, instance eq True
namedtuple size 56, hashable True, instance eq True
dataclass size 56, hashable True, instance eq True
class size 56, hashable True, instance eq False


## set

存储可散列的对象，最重要的特点是唯一性

In [169]:
suits_set = {"♠️", "♥️", "♣️", "♦️"}
suits_set = set(["♠️", "♥️", "♣️", "♦️", "♦️"])
print(suits_set)
suits_set.add("♦️")
print(suits_set)

{'♣️', '♠️', '♥️', '♦️'}
{'♣️', '♠️', '♥️', '♦️'}


In [ ]:
class A:
    def __init__(self) -> None:
        pass
cls_set = {A(), A()}
print(len(cls_set))

### 从列表里去重，并保持原有顺序？

### `in`对应的是存在`hash`值，而`==`对应的是`__eq__`

In [189]:
class A:
    def __init__(self, value=1) -> None:
        self.value = value
    def __eq__(self, __o: object) -> bool:
        if type(self) == type(__o):
            return self.value == __o.value
        return False
    def __hash__(self) -> int:
        return id(self)
    def __repr__(self) -> str:
        return str(self.value)
a, b = A(2), A(3)
print(a == b)
cls_set = {b, a}
c = A(2)
print(a in cls_set)
print(c in cls_set)
print(cls_set)

False
True
False
{3, 2}


# bytes
以一个字节（uint8）的标准形式，依赖编解码器（通过什么表格映射解释字节，本文略）。在SSDV中，常见的场景是向服务器发送消息，需要以固定接口格式组装消息。假设一个简单的场景，服务器维护了玩家的最高分，当玩家运行出结果后，会向服务器发送：
```[C]
typedef struct {
    UINT32 tag;     # 消息的类型
    UINT32 len;     # 消息的长度
    UINT8 value[];  # 消息的内容
} MsgHeaderStru;

typedef enum {
    SERVER_RECV_USER_SCORE = 0x0001,
} MsgTagEnum;

#define MAX_USER_ID_LEN 10
typedef struct {
    UINT64 usrId; # 用户id，如s00123456，则为123456，如果为500123456，也足够
    INT8 score;    # 如果sum < 21，则为sum，反之为21 - sum，该值的精度为0.5，即1对应0.5
    UINT8 cardNumber; # score一致时，通过cardNumber比较
    UINT8 rsv[6];
} MsgUserScoreStru;
```
如何组装该消息？

## `struct`消息组装
* 类型

|格式码|	C 类型|	python 类型|
| --- | --- | --- |
|c | char |	bytes of length 1 |
|b | int8|	int|
|B | uint8| int|
|?|_Bool|bool|
|h | int16|int|
|i|int32|int|
|q|int64|int|
|d|double|double|
* 字节序

|字节序|顺序|
| --- | --- |
|<|小端|
|>|大端|
|!|网络序(大端)|
无符号是有符号的大写
* 组包：`struct.pack(formats, value1, value2, ...)`
* 大小：`struct.calcsize(formats)`
* 解包：`struct.unpack(formats, value1, value2, ...)`

In [206]:
import struct
"""
#define MAX_USER_ID_LEN 10
typedef struct {
    UINT64 usrId; 
    INT8 score;
    UINT8 cardNumber; 
    UINT8 rsv[6];
} MsgUserScoreStru;
"""
msg_user_score = ">QbBBBBBBB"
print(struct.calcsize(msg_user_score))
head = struct.pack(">II", 0x0001, struct.calcsize(msg_user_score))
print(head)
msg_body = struct.pack(msg_user_score, 123456, 40, 5, 0, 0, 0, 0, 0, 0)
msg = head + msg_body

16
b'\x00\x00\x00\x01\x00\x00\x00\x10'


In [211]:
tag, len, usrid, score, card_num, *_ = struct.unpack(">II"+msg_user_score.replace(">", ""), msg)
print(tag, len, usrid, score, card_num)

1 16 123456 40 5


## ctypes中的Structure

In [250]:
from ctypes import Structure, BigEndianStructure, c_uint64, c_uint32, c_uint8, c_int8, sizeof
class MsgHeaderStru(BigEndianStructure):
    _fields_ = [
        ("tag", c_uint32),
        ("len", c_uint32),
    ]

class MsgUserScoreStru(BigEndianStructure):
    _fields_ = [
        ("usrId", c_uint64),
        ("score", c_int8),
        ("cardNumber", c_uint8),
        ("rsv", c_uint8 * 6),
    ]

msg_header = MsgHeaderStru(tag = 1, len = sizeof(MsgUserScoreStru))
rsv_tmp = [0] * 6
# 默认使用0初始化，仅做list初始化说明
msg_user_score = MsgUserScoreStru(usrId=123456, score=40, cardNumber=5, rsv=(c_uint8 * 6)(*rsv_tmp))
bytes(msg_header)

b'\x01\x00\x00\x00\x10\x00\x00\x00'